In [41]:
## caged_brach
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

from paths import *
from scpts import manipulation
from scpts import io
from scpts import caged_novo

from basedosdados import Storage, Table, Dataset
import basedosdados as bd

import glob
import shutil


### Download Dados
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED_AJUSTES/
 - ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/

### Get Urls

In [7]:
download_dict = caged_novo.get_download_links()

estabelecimentos
movimentacoes


### DOWNLOAD E TRATAMENTO

- download
- extrair
- load
- padroniza nome das colunas
- limpa
- salva, formato ano=xxxx/mes=xx/sigla_uf=XX
- upload bd
- deleta arquivos

In [15]:
today = datetime.strftime(datetime.today(), "%Y-%m-%d")


RAW_PATH = "../data/caged_novo/raw/"
CLEAN_PATH = "../data/caged_novo/clean/"

# GET MUNICIPIOS FROM BD
print("====== ", today, " ======")
query = """
SELECT 
    sigla_uf,
    id_municipio,
    id_municipio_6
FROM `basedosdados.br_bd_diretorios_brasil.municipio` 
"""

municipios = bd.read_sql(query, billing_project_id="basedosdados-dev")
print("\n")

======  2021-10-05  ======


Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5570/5570 [00:00<00:00, 9711.31rows/s]

In [30]:
def upload_to_raw(tipo, save_raw_path):
    if tipo == "estabelecimentos":
        st = bd.Storage(table_id="microdados_estabelecimentos", dataset_id="br_me_caged")
    else:
        st = bd.Storage(table_id="microdados_movimentacoes", dataset_id="br_me_caged")
        
    st.upload(
        path=save_raw_path,
        mode='raw',
        if_exists='replace'
    )

In [34]:
# deleta pasta
if os.path.isdir(CLEAN_PATH):
    shutil.rmtree(CLEAN_PATH)
if os.path.isdir(RAW_PATH):
    shutil.rmtree(RAW_PATH)

print("Get download links")
download_dict = caged_novo.get_download_links()
print("\n")

for tipo in list(download_dict.keys()):

    download_opt = caged_novo.get_filtered_download_dict(
        tipo=tipo, download_dict=download_dict, bucket_name="basedosdados-dev"
    )

    print(f"Filter download links {tipo}")

    trigger, download_opt = caged_novo.get_trigger_and_download_opt(download_opt, tipo)

    if trigger:
        for year_month_path in list(download_opt.keys()):
            print(tipo, ": ", year_month_path)
            ## download data
            save_path = RAW_PATH + f"{tipo}/" + year_month_path
            caged_novo.download_data(save_path, download_opt[year_month_path])
            
            #create some vars
            file_name = download_opt[year_month_path].split("/")[-1].split(".")[0]
            file_path = RAW_PATH + f"{tipo}/" + year_month_path
            
#             caged_novo.upload_to_raw(tipo=tipo, save_raw_path=f"{save_path}{file_name}.7z")


            # extrai arquivo
            caged_novo.extract_file(file_path, file_name, save_rows=None)

            # load e organiza os dados
            df = caged_novo.rename_add_orginaze_columns(file_path, file_name, tipo, municipios)

            # salva no formato de particao
            save_clean_path = CLEAN_PATH + f"{tipo}/"
            caged_novo.creat_partition(df, save_clean_path, year_month_path)

            # upload basedosdados
#             caged_novo.upload_to_bd(tipo, save_clean_path)

            # deleta pasta
#             shutil.rmtree(CLEAN_PATH + f"{tipo}/")
#             shutil.rmtree(RAW_PATH + f"{tipo}/")

    print("\n")


Get download links
estabelecimentos
movimentacoes


Filter download links estabelecimentos
nothing to download


Filter download links movimentacoes
movimentacoes :  2020/1/




Downloading: 100%|██████████████████████| 5570/5570 [00:00<00:00, 9060.92rows/s]


,sigla_uf,id_municipio,id_municipio_6
0,RO,1100015,110001
1,RO,1100023,110002
2,RO,1100031,110003
3,RO,1100049,110004
4,RO,1100056,110005
...,...,...,...
5565,GO,5222005,522200
5566,GO,5222054,522205
5567,GO,5222203,522220
5568,GO,5222302,522230


### Create table and fill metadata

In [35]:
tb = bd.Table(table_id = 'microdados_movimentacoes',dataset_id = 'br_me_caged')
# st = bd.Storage('br_me_caged','microdados_movimentacoes',)
# st.delete_table(bucket_name='basedosdados-dev')
tb.create(path='../data/caged_novo/clean/movimentacoes/',
          if_table_config_exists='pass',
          if_storage_data_exists='replace',
          if_table_exists='replace'
         )

# tb.update_columns('https://docs.google.com/spreadsheets/d/1ihyOCSkaarmR3uMHj8bmP9tv-OChVeL2LkEWaGqxp7w/edit#gid=787251136')

tb.publish(if_exists='replace')

Uploading files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:51<00:00,  1.91s/it]


In [54]:
tb = bd.Table(table_id = 'microdados_estabelecimentos',dataset_id = 'br_me_caged')
st = bd.Storage('br_me_caged','microdados_estabelecimentos',)
st.delete_table(bucket_name='basedosdados-dev')
# tb.create(path='../data/caged_novo/clean/estabelecimentos/', 
#           if_table_config_exists='pass',
#           if_storage_data_exists='replace',
#           if_table_exists='replace' 
#          )

# tb.update_columns('https://docs.google.com/spreadsheets/d/1-XQM9mhlJ3-t10c7cPpwOOK24hKz0u3qR6m04I1PoR4/edit#gid=1885260190')

# tb.publish(if_exists='replace')

In [55]:
## fill columns description


In [46]:
### SQL

In [3]:
tb = bd.Table(table_id='dicionario',dataset_id='br_me_caged')
tb.create(path='/Users/m/Downloads/dicionario.csv',
          if_table_config_exists='replace',
          if_storage_data_exists='replace',
          if_table_exists='replace'
         )

Uploading files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.23it/s]


In [4]:
tb.publish()

## VIZ

In [42]:
query = """
SELECT * FROM `basedosdados-projetos.br_me_caged.desligamento_por_morte`
"""
df = bd.read_sql(query, billing_project_id='basedosdados-dev')

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 491442/491442 [00:33<00:00, 14679.08rows/s]


In [76]:
br = df.groupby(by=['ano','mes'], as_index=False).sum()
date = pd.to_datetime(br['ano'].astype(str) + '-' + br['mes'].astype(str), format='%Y-%m').apply(lambda x: x.strftime('%Y-%m')) 
br.insert(0,'date',date)
br = br[br['ano']>=2000].drop(columns=['ano','mes'])
br.to_clipboard(index=False)

In [77]:
br

,date,desligamento_morte
0,2007-01,3226
1,2007-02,2804
2,2007-03,3106
3,2007-04,3033
4,2007-05,3315
5,2007-06,3320
6,2007-07,3399
7,2007-08,3260
8,2007-09,3211
9,2007-10,3409
